지금까지는 훈련 세트에서 모델을 훈련하고 테스트 세트에서 모델을 평가했다.<br/>
테스트 세트에서 얻은 점수를 통해 일반화 성능을 가늠해 본 것이다.<br/> 그런데 테스트 세트를 이용해 성능을 자꾸 확인하다 보면 점점 테스트 세트에 적합한 모델이 된다는 문제가 생긴다.<br/>
사실 테스트 세트로 일반화 성능을 올바르게 예측하기 위해선 가능한 테스트 세트를 사용하지 말아야 한다. 모델을 만들고 마지막에 딱 한 번만 사용하는것이 이상적이다.<br/>
그러나 테스트 세트를 사용하지 않으면 이 모델이 과대적합인지, 과소적합인지 판단하기 어렵다. 이를 해결하기위한 간단한 방법은 훈련세트를 다시 나누어 검증 세트(validation set)를 만드는 것이다.

In [2]:
#pandas를 이용해 데이터를 읽어옴
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine-date')   

In [3]:
#데이터를 넘파이 배열로 변환
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
#데이터를 훈련 세트와 테스트 세트로 나눔
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)     #전체 데이터의 20%를 테스트 세트로 사용

In [6]:
#훈련 세트를 나누어 검증세트를 만듬
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
print(sub_input.shape, val_input.shape)     #잘 나누어 졌는지 크기 확인

(4157, 3) (1040, 3)


In [7]:
#훈련세트와 검증세트를 이용해 모델을 만들고 평가
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))      #점수 확인 결과 과대적합

0.9971133028626413
0.864423076923077


검증 세트를 이용할 경우, 기존의 훈련세트를 나누었기 때문에 훈련세트의 크기가 작아진다.<br/>
통상적으로 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어 지기 때문에 이 현상이 탐탁치 않다. 그렇다고 검증세트를 너무 작게 만들면 검증 점수가 불안정할 것이다.<br/>
이럴 때 안정적인 검증 점수와 더 많은 훈련 데이터를 얻는데 사용하는 방법이 __교차 검증(cross validation)__이다.<br/>
교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러번 반복한 뒤 그 점수를 평균하여 최종 검증 점수를 얻는다.<br/>
예를 들어 k-폴드 교차 검증(k-fold cross validation)은 훈련세트를 k개의 그룹으로 나눠서 그 중 하나를 검증세트로 하여 교차 검증을 진행한다.<br/>
우리는 사이킷런의 cross_validate()라는 함수를 이용해 교차검증을 시행할 수 있다.

In [16]:
#교차검증
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)      #검증세트를 따로 나누지 않고 훈련세트 전체를 전달
print(scores)
#cross_validate()는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환한다.
##첫 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미하며 cross_validate()는 기본적으로 5-폴드 교차 검증을 수행하기 때문에 각 키마다 5개의 숫자를 가진다.
###교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 편균하여 얻을수 있다. 이름과 달리 검증 폴드의 점수임에 유의하자.

{'fit_time': array([0.01208115, 0.00657821, 0.00678849, 0.00685477, 0.00644684]), 'score_time': array([0.00068164, 0.0004344 , 0.00052428, 0.00063944, 0.00051141]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [11]:
#test_score의 값만을 원한다면 cross_val_score()함수를 이용할 수도 있다.
from sklearn.model_selection import cross_val_score

Scores = cross_val_score(dt, train_input, train_target)
print(Scores)

[0.86923077 0.84615385 0.87680462 0.84889317 0.83541867]


In [17]:
#검증 폴드의 점수들을 평균함
import numpy as np

print(np.mean(scores['test_score']))
#혹은 print(np.mean(Scores))

0.855300214703487


전체 데이터를 섞은 후 훈련데이터를 준비해주는 train_test_split()과 달리 cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다.<br/>
때문에 교차검증을 할 때 훈련 세트를 섞으려면 분할기(splitter)를 지정해야 한다.<br/>
사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해 준다. cross_validate()함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용하고 분류 모델일 경우 타킷 클래스를 골고루 나누기 위해 StratifiedKFold를 사용한다.

In [18]:
#즉 앞서 수행한 교차 검증은 다음과 동일하다.
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [19]:
#만약 훈련세트를 섞은 후 10-폴드 교차 검증을 수행하려면 다음과 같이 작성해야한다.

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)      #n_splits 매개변수는 몇 폴드 교차 검증을 할지 정함.
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


교차 검증에 대해 이해 했으니 이제 결정 트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아보자.<br/>
머신러닝 모델이 학습하는 파라미터를 모델 파라미터라고 부르는 반면, 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터를 하이퍼파라미터라고 부른다. 머신러닝 라이브러리를 사용할 때 이런 하이퍼파라미터는 모두 클래스나 메서드의 매개변수로 표현된다.<br/>
이런 하이퍼파라미터를 튜닝하려면 매개변수를 직접 바꿔가면서 모델을 훈련하고 교차 검증을 수행하여야하는데 매개변수가 많아질수록 문제가 복잡해져 직접 구현하기 어렵다.<br/>
다행히도 사이킷런에서 제공하는 그리드 서치(Grid Search)를 사용하면 이를 수행할 수 있다.<br/>
다음은 GridSearchCV()의 예시이다.

In [24]:
#탐색할 매개변수로 min_impurity_decrease를 지정하고 탐색할 값의 리스트를 딕셔너리로 만든다.
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}        #min_impurity_decrease에 주어진 값들을 넣어 검사
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)     #GridSearchCV클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체 생성. n_jobs값은 사용할 CPU 코어 개수
gs.fit(train_input, train_target)     #GridSearchCV의 cv 매개변수 기본값은 5이므로 총 5x5개의 모델을 훈련함
dt = gs.best_estimator_     #검증 점수가 가장 높은 모델은 best_estimator_ 속성에 저장됨
print(dt.score(train_input, train_target))
print(gs.best_params_)      #최적의 매개변수는 best_params_ 속성에 저장됨
print(gs.cv_results_['mean_test_score'])      #각 매개변수에 대한 교차 검증의 평균 점수는 cv_results의 mean_test_score속성에 저장됨
best_index = np.argmax(gs.cv_results_['mean_test_score'])     #넘파이 argmax() 함수를 이용하면 가장 큰 값의 인덱스를 추출할 수 있음
print(gs.cv_results_['params'][best_index])     #인덱스를 이용해 최적의 매개변수 출력

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


더 많은 매개변수로 예시를 들어보자.

In [27]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001), 'max_depth' : range(5, 20, 1), 'min_samples_split' : range(2, 100, 10)}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [29]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


위의 예시와는 달리 매개변수의 값의 범위나 간격을 미리 정하기 어려운 경우가 있을 수 있다.<br/>
그럴 때 사용하면 좋은 것이 랜덤 서치(random search)다. 랜덤 서치는 매개변수의 목록을 전달하는 대신 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달합니다.

In [31]:
from scipy.stats import uniform, randint      #uniform와 randint 클래스 모두 주어진 범위에서 고르게 값을 뽑는다.(균등 분포에서 샘플링한다.) randint는 정수값을, uniform은 실숫값을 뽑는다.

rgen = randint(0, 10)
rgen.rvs(10)      #0~10사이의 정수를 10번 뽑음

array([9, 7, 3, 2, 2, 1, 6, 1, 4, 5])

In [33]:
np.unique(rgen.rvs(1000), return_counts=True)     #0~10사이의 정수를 1000번 뽑아 각 숫자의 개수를 셈

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 98, 114,  77,  94,  98, 101,  98, 107,  98, 115]))

In [34]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.86276429, 0.57296415, 0.71431921, 0.92769333, 0.84016911,
       0.20158937, 0.00875381, 0.0963483 , 0.44266387, 0.50891618])

In [36]:
parmas = {'min_impurity_decrease' : uniform(0.0001, 0.001), 'max_depth' : randint(20, 50), 'min_samples_split' : randint(2, 25), 'min_samples_leaf' : randint(1, 25)}
#min_samples_leaf 매개변수는 리프 노드가 되기 위한 최소 샘플의 개수이다. 어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않는다.

In [37]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), parmas, n_iter=100, n_jobs=-1, random_state=42) #샘플링 횟수는 사이킷런의 램덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정한다.
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [38]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [39]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [40]:
dt = gs.best_estimator_     #최적의 모델은 best_estimator_속성에 저장
print(dt.score(test_input, test_target))      #테스트 점수는 검증 점수보다 조금 낮은것이 일반적.

0.86
